In [1]:
import psi4
import numpy as np
import sys
sys.path.append('../lib')
from P4toC4_aux import *
from SO_aux import SymOrbs

In [2]:
#BASIS='STO-3G'
BASIS='def2-SVP'
#BASIS='CC-PVDZ'

In [3]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", BASIS)
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

mol = psi4.geometry("""
0 1
H        -1.14356293    -1.24134665     0.75589061
H        -1.14356293    -1.24134665    -0.75589061
HE       -1.81408822     0.52541937     0.00000000
HE        2.38996911     0.09970391     0.00000000
units Bohr
no_reorient
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-6.6393832862296485

In [5]:
# Print a new basis set in GENBAS format
basisset=wf.basisset()
#print(basisset.genbas())

In [6]:
mol=wf.molecule()
ptgr=mol.point_group()
print(f'{ptgr.symbol()}: order = {ptgr.order()}')
n_irrep=wf.nirrep()
g=wf.nmopi()
n_mo_pi=g.to_tuple()
print('MOs per irrep', n_mo_pi)

cs: order = 2
MOs per irrep (13, 7)


In [7]:
# Psi4 MOs in SO basis
Ca=wf.Ca()
#Cb=np.array(wf.Cb())
Ca.shape

((13, 13), (7, 7))

In [8]:
#BASIS='PVDZ'
C4_cs = read_oldmos('../Cfour/SYM/Cs/OLDMOS.'+BASIS, n_mo_pi)

reading orbitals from ../Cfour/SYM/Cs/OLDMOS.def2-SVP


In [10]:
sym=1
Cp=Ca.nph[sym]
Cc=C4_cs[sym]
naos=n_mo_pi[sym]
mo=0
print('          Psi4         Cfour')
for k in range(naos):
    print(f'{k+1:3d}  {Cp[k,mo]:10.6f} {Cc[k,mo]:10.6f}')

          Psi4         Cfour
  1    0.237326   0.167815
  2    2.490836   1.761287
  3   -0.005293  -0.003890
  4   -0.005501   0.012260
  5    0.017338  -0.003743
  6    0.016914   0.016914
  7    0.009296   0.009296


In [11]:
#
#  comparison Psi4-MOs and Cfour-MOs in their SO representation
#
for i in range(wf.nirrep()):
    print(np.max(Ca.nph[i]-C4_cs[i]))

0.2409195646257175
0.47564116458819083


### Get symmetry ordered MOs in AO basis 

In [11]:
C_AO_pi=AO_MO_by_irrep(wf, verbose=0)
C_AO_pi.shape

((20, 13), (20, 7))

### Transform to Cfour-SO basis 

In [14]:
Ls=wf.aotoso()
print(Ls.shape)

((20, 13), (20, 7))


In [15]:
for isym in range(ptgr.order()):
    print(f'\nSymmetry {isym}')
    SOs=SymOrbs(Ls.nph[isym], order=ptgr.order())
    SOs.print()


Symmetry 0
 0   0.70711(  0)   0.70711(  5)
 1   0.70711(  1)   0.70711(  6)
 2   0.70711(  2)  -0.70711(  7)
 3   0.70711(  3)   0.70711(  8)
 4   0.70711(  4)   0.70711(  9)
 5   1.00000( 10)
 6   1.00000( 11)
 7   1.00000( 13)
 8   1.00000( 14)
 9   1.00000( 15)
10   1.00000( 16)
11   1.00000( 18)
12   1.00000( 19)

Symmetry 1
 0   0.70711(  0)  -0.70711(  5)
 1   0.70711(  1)  -0.70711(  6)
 2   0.70711(  2)   0.70711(  7)
 3   0.70711(  3)  -0.70711(  8)
 4   0.70711(  4)  -0.70711(  9)
 5   1.00000( 12)
 6   1.00000( 17)


In [16]:
ao_map, scale = basis_mapping(wf.basisset(), verbose=0)
ao_map

array([ 0,  1,  4,  2,  3,  5,  6,  9,  7,  8, 10, 11, 14, 12, 13, 15, 16,
       19, 17, 18])

In [17]:
basisset=wf.basisset()
mapping, scale = basis_mapping(wf.basisset(), verbose=0)
C_SO_pi0=AO_to_SO(C_AO_pi, wf)
C_SO_pir=AO_to_Cfour_SO(C_AO_pi, wf, mapping, scale)
print(C_SO_pir.shape,C_SO_pir.shape)

((3, 3), (1, 1)) ((3, 3), (1, 1))


In [20]:
sym=1
Cp0=C_SO_pi0.nph[sym]
Cpr=C_SO_pir.nph[sym]
Cc=C4_cs[sym]
naos=n_mo_pi[sym]
mo=1
print('        Psi4-org  Psi4-reord   Cfour')
for k in range(naos):
    print(f'{k+1:3d}  {Cp0[k,mo]:10.4f} {Cpr[k,mo]:10.4f} {Cc[k,mo]:10.4f}')

        Psi4-org  Psi4-reord   Cfour
  1      0.9656     0.0000     0.9656
  2     -1.0351     0.0000    -1.0351


In [19]:
Cpr

array([[0., 0.],
       [0., 0.]])

In [21]:
for sym in range(n_irrep):
    d=np.max(C_SO_pi.nph[sym]-C4_cs[sym])
    print(f'{sym}   delta={d:10.3e}')

0   delta= 1.593e+00
1   delta= 6.965e-08


In [24]:
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)

In [25]:
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)